In [19]:
#Importing necessary libraries
import pandas as pd
import pymysql

In [20]:
#MySQL Connection setup
connection = pymysql.connect(
    host = 'localhost',
    user ='root',
    password = 'Refolin@2293',
    autocommit =True
)
cursor = connection.cursor()

In [21]:
#Getting the cursor details
print(cursor)

In [22]:
#Creating the "BigMart" database
cursor.execute("CREATE DATABASE IF NOT EXISTS BigMart;")
cursor.execute("USE BigMart;")

0

In [23]:
#Loading the 3 xml files
df_item = pd.read_xml("df_item.xml")
df_outlet=pd.read_xml("df_outlet.xml")
df_sales=pd.read_xml("df_sales.xml")

In [24]:
# Creating function for creating tables 
def create_table_from_df(df, table_name):
    cols = []
    for col in df.columns:
        dtype = df[col].dtype
        if 'int' in str(dtype):
            sql_type = 'INT'
        elif 'float' in str(dtype):
            sql_type = 'FLOAT'
        else:
            sql_type = 'VARCHAR(255)'
        cols.append(f"{col} {sql_type}")

    columns_sql = ", ".join(cols)
    create_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql})"
    cursor.execute(create_sql)

In [25]:
#Creating table in database
create_table_from_df(df_item, "item_info")
create_table_from_df(df_outlet, "outlet_info")
create_table_from_df(df_sales, "sales_info")

In [26]:
def insert_data(df,table_name):
    df = df.where(pd.notnull(df), None)
    cols=",".join([f"`{col}`" for col in df.columns])
    placeholders = ",".join(["%s"] * len(df.columns))
    insert_sql = f"INSERT INTO `{table_name}` ({cols}) VALUES ({placeholders})"
    data = [tuple(row) for row in df.itertuples(index = False, name = None)]
    cursor.executemany(insert_sql, data)

In [27]:
#Inserting the data into MYSQL Tables
insert_data(df_item, 'item_info')
insert_data(df_outlet, 'outlet_info')
insert_data(df_sales, 'sales_info')


In [28]:
# Closing the connection 
cursor.close()
connection.close()